## 参考文献

https://users.aalto.fi/~vanvlm1/mne-rsa/auto_examples/plot_sensor_level.html


http://www.pymvpa.org/examples.html#chap-examples
http://www.pymvpa.org/tutorial_eventrelated_searchlight.html#chap-tutorial-eventrelated-searchlight
http://www.pymvpa.org/generated/mvpa2.measures.searchlight.Searchlight.html#mvpa2.measures.searchlight.Searchlight

several toolboxes have been developed that
implement the methods described in the rest of this paper; the PyMVPA toolbox (Hanke, Halchenko, Sederberg, Hanson, et al., 2009; www.pymvpa.org) handles both fMRI and M/EEG data using the open-source Python language (Hanke, Halchenko, Sederberg, Olivetti, et al., 2009)



https://mne.tools/stable/auto_examples/decoding/linear_model_patterns.html#ex-linear-patterns


In [1]:
import mne
from mne import io, EvokedArray
from mne.datasets import sample
from mne.decoding import Vectorizer, get_coef

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# import a linear classifier from mne.decoding
from mne.decoding import LinearModel
import numpy as np

In [2]:
list_epochs_all = list()

sub_ids = [

    "pre4008_y","pre4009_y","pre4010_y","pre4011_y","pre4012_y","pre4013_y",
    "pre4014_y","pre4015_y","pre4017_y",
    "pre4021_y","pre4022_y","pre4023_y","pre4026_y","pre4027_y","pre4028_y","pre4029_y","pre4030_y",


]

file_path = 'E:\实验\阅读困难\数据处理\pre4\data\\6epoch_clean_RSA\\'

for sub_id in sub_ids:
    data_path = file_path + sub_id + '_RSA-epo.fif'
    epochs_all = mne.read_epochs(fname=data_path)
    #做decoding要考虑被试内每个条件试次一样，如果被试间不一样可以用循环来解决。
    epochs_all.equalize_event_counts(method='mintime')
    list_epochs_all.append(epochs_all)

# 如果只想看某一部分的脑区,配合更改ndarray的维度
# channels_nieye = ['M1','T7','P7','TP7','M2','T8','P8','TP8','CP5','CP6','C5','C6']
# epochs_nieye = epochs_all.pick(picks=channels_nieye)

'''
思路，先求ERP，然后再给ndarray，然后再计算RDM
'''

Reading E:\实验\阅读困难\数据处理\pre4\data\6epoch_clean_RSA\pre4008_y_RSA-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
1275 matching events found
No baseline correction applied
0 projection items activated
Dropped 59 epochs: 100, 101, 103, 113, 118, 119, 131, 135, 136, 137, 139, 141, 189, 192, 193, 385, 400, 402, 403, 420, 579, 582, 585, 740, 750, 901, 905, 907, 923, 927, 1053, 1057, 1058, 1059, 1072, 1075, 1076, 1077, 1170, 1172, 1173, 1174, 1175, 1177, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 1191, 1192, 1193, 1194
Reading E:\实验\阅读困难\数据处理\pre4\data\6epoch_clean_RSA\pre4009_y_RSA-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
1007 matching events found
No baseline correction applied
0 projection items activated
Dropped 63 epochs: 0, 2, 4, 7, 9, 12, 13, 15, 133,

'\n思路，先求ERP，然后再给ndarray，然后再计算RDM\n'

In [3]:
'''
先整理单个的被试看看

data的结构要求： [n_subs, n_trials, n_chls, n_ts]
label的结构要求： [n_subs, n_trials]

'''

conds = ['101', '102', '103', '104', '111', '112', '113', '114', '121', '122', '123', '124', '131', '132', '133', '134', ]

num_conds = len(conds)

#初始值
subindex = 0
list_subdata = list()
list_sublabel = list()
n_subs = len(sub_ids)
n_ts = 191
accs_fuyin_allsubs = np.zeros([n_subs, n_ts])
accs_yindiao_allsubs = np.zeros([n_subs, n_ts])

In [ ]:
for m in range(len(sub_ids)):
    #单个被试的数据
    epochs_forlen = list_epochs_all[m][conds[0]]
    # 每个被试的trial数量是不一样的
    # trial数不能放在第一位，第一位是condition
    num_trials = len(epochs_forlen.events[:,1])
    subdata = np.zeros([num_conds, num_trials, 62,  1001], dtype=np.float32)
    sublabel = np.zeros([0,0])

    for i in range(len(conds)):
        epochs = list_epochs_all[m][conds[i]]
        #做decoding，这里就不能是平均了吧。
        # 这里get_data()还会把所有的channel都选上，你需要pick一下,只要eeg，不要眼动数据
        data = epochs.get_data(picks='eeg')
        label_cond = epochs.events[:,2]
        #每种条件的cond
        sublabel = np.append(sublabel,label_cond)
        sublabel = np.reshape(sublabel, [1,len(sublabel[:])])
        subdata[i] = data
        # 现在是[con, trial, chan, ts]
        # [n_cons, n_subs, n_trials, n_chls, n_ts]需要这样的结构
        # 需要中间插入一个subs
    subdata = np.reshape(subdata, [num_conds, 1, num_trials, 62, 1001])
    # 在某个维度上面拉平
    # 把condition的维度乘到 trial上面？
    data_decode = np.reshape(subdata,[1, num_conds * num_trials,  62, 1001])
    list_subdata.append(data_decode)
    list_sublabel.append(sublabel)

    label_original = sublabel
    label_yindiao = ((label_original-100)%10).astype(int)
    accs_yindiao = tbyt_decoding_kfold(data_decode,label_yindiao, n=4, time_win=50,time_step=5, nfolds=5, nrepeats=10, navg=10)
    accs_yindiao_allsubs[i] = accs_yindiao


In [ ]:
m = 0
#单个被试的数据
epochs_forlen = list_epochs_all[m][conds[0]]
# 每个被试的trial数量是不一样的
# trial数不能放在第一位，第一位是condition
num_trials = len(epochs_forlen.events[:,1])
subdata = np.zeros([num_conds, num_trials, 62,  1001], dtype=np.float32)
sublabel = np.zeros([0,0])

for i in range(len(conds)):
    epochs = list_epochs_all[m][conds[i]]
    #做decoding，这里就不能是平均了吧。
    # 这里get_data()还会把所有的channel都选上，你需要pick一下,只要eeg，不要眼动数据
    data = epochs.get_data(picks='eeg')
    label_cond = epochs.events[:,2]
    #每种条件的cond
    sublabel = np.append(sublabel,label_cond)
    sublabel = np.reshape(sublabel, [1,len(sublabel[:])])
    subdata[i] = data
    # 现在是[con, trial, chan, ts]
    # [n_cons, n_subs, n_trials, n_chls, n_ts]需要这样的结构
    # 需要中间插入一个subs

In [17]:
print(sublabel)
print(sublabel.shape)
subdata.shape

[[101. 101. 101. ... 134. 134. 134.]]
(1, 1216)


(16, 76, 62, 1001)

In [18]:
meg_epochs = list_epochs_all[0].copy().pick_types(meg=False, eeg=True)
labels = sublabel.flatten()
meg_data = meg_epochs.get_data().reshape(1216, -1)



NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


In [19]:
print(len(sublabel))

1


In [ ]:

clf = LogisticRegression(solver="liblinear")  # liblinear is faster than lbfgs
scaler = StandardScaler()

# create a linear model with LogisticRegression
model = LinearModel(clf)

# fit the classifier on MEG data
X = scaler.fit_transform(meg_data)
model.fit(X, labels)

# Extract and plot spatial filters and spatial patterns
for name, coef in (("patterns", model.patterns_), ("filters", model.filters_)):
    # We fitted the linear model onto Z-scored data. To make the filters
    # interpretable, we must reverse this normalization step
    coef = scaler.inverse_transform([coef])[0]

    # The data was vectorized to fit a single model across all time points and
    # all channels. We thus reshape it:
    coef = coef.reshape(len(epochs.ch_names), -1)

    # Plot
    evoked = EvokedArray(coef, epochs.info, tmin=epochs.tmin)
    fig = evoked.plot_topomap()
    fig.suptitle(f"MEG {name}")